In [1]:
import pandas as pd

In [4]:
!pip install cryptocode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.6 MB/s eta 0:00:0000:01


In [1]:
!pip install cryptography

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 31.1 MB/s eta 0:00:0000:010:01


In [2]:
from cryptography.fernet import Fernet 
message = "hello geeks"
  
key = Fernet.generate_key() 
  
  
fernet = Fernet(key) 
  
encMessage = fernet.encrypt(message.encode()) 
  
print("original string: ", message) 
print("encrypted string: ", encMessage) 
  
decMessage = fernet.decrypt(encMessage).decode() 
  
print("decrypted string: ", decMessage) 

original string:  hello geeks
encrypted string:  b'gAAAAABlyshKT8QNVNlSS4ndsAsDPIyVoqh8EU14ZtR6Kg5l9HreVE44xEQZ40YaIS37XUh-4519cO0aohhrBoP95ayo6avE9g=='
decrypted string:  hello geeks


In [10]:
import requests
import pandas as pd
from datetime import datetime
import cryptocode
import json
import sqlite3

timestamp = datetime.now().strftime('%Y-%m-%d')

url_api = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados/35/municipios'

response = requests.get(url_api)

dados = response.json()

dados_list = []

if dados:
    for resultado in dados:
        id = resultado['id']
        nome = resultado['nome']
        id_mun = resultado['microrregiao']['id']
        nm_mun = resultado['microrregiao']['nome']
        id_mcr = resultado['microrregiao']['mesorregiao']['id']
        nm_mcr = resultado['microrregiao']['mesorregiao']['nome']
        id_uf = resultado['microrregiao']['mesorregiao']['UF']['id']
        nm_uf = resultado['microrregiao']['mesorregiao']['UF']['nome']
        sigla_uf = resultado['microrregiao']['mesorregiao']['UF']['sigla']
        id_reg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['id']
        nm_reg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['nome']
        sigla_rg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['sigla']
        rgimd_id = resultado['regiao-imediata']['id']
        rgimd_nm = resultado['regiao-imediata']['nome']
        rgint_id = resultado['regiao-imediata']['regiao-intermediaria']['id']
        rgint_nm = resultado['regiao-imediata']['regiao-intermediaria']['nome']
        rguf_id = resultado['regiao-imediata']['regiao-intermediaria']['UF']['id']
        rguf_sgl = resultado['regiao-imediata']['regiao-intermediaria']['UF']['sigla']
        rguf_nm = resultado['regiao-imediata']['regiao-intermediaria']['UF']['nome']
        rgrg_id = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['id']
        rgrg_sgl = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['sigla']
        rgrg_nm = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['nome']
        
        dados_list.append({
            'id': id,
            'nome': nome,
            'id_mun': id_mun,
            'nm_mun': nm_mun,
            'id_mcr': id_mcr,
            'nm_mcr': nm_mcr,
            'id_uf': id_uf,
            'nm_uf': nm_uf,
            'sigla_uf': sigla_uf,
            'id_reg': id_reg,
            'nm_reg': nm_reg,
            'sigla_rg': sigla_rg,
            'rgimd_id': rgimd_id,
            'rgimd_nm': rgimd_nm,
            'rgint_id': rgint_id,
            'rgint_nm': rgint_nm,
            'rguf_id': rguf_id,
            'rguf_sgl': rguf_sgl,
            'rguf_nm': rguf_nm,
            'rgrg_id': rgrg_id,
            'rgrg_sgl': rgrg_sgl,
            'rgrg_nm': rgrg_nm,
            'dtigtao': timestamp
        })

df = pd.DataFrame(dados_list)

# Converte o DataFrame em JSON e depois em uma string
mensagem = df.to_json()

chave = "423338233093093"

MensagemCriptografada = cryptocode.encrypt(mensagem, chave)

caminho = '/workspaces/api-geral/GEO/dados/'
arquivo = 'dados_criptografados.txt'

# Salvar a mensagem criptografada em um arquivo
with open(f"{caminho}{arquivo}", "w") as file:
    file.write(MensagemCriptografada)
    print('arquivo criptogrado salvo!')
    print(f'Caminho: {caminho}')

arquivo criptogrado salvo!
Caminho: /workspaces/api-geral/GEO/dados/


In [12]:
arquivo_cript = '/workspaces/api-geral/GEO/dados/dados_criptografados.txt'

# Ler a mensagem criptografada do arquivo
with open(arquivo_cript, "r") as file:
    mensagem_criptografada = file.read()

# Descriptografar a mensagem
mensagem_descriptografada = cryptocode.decrypt(mensagem_criptografada, chave)

# Converter a mensagem descriptografada de volta para DataFrame
df_descriptografado = pd.read_json(mensagem_descriptografada)

# Salvar os dados descriptografados em um banco de dados SQLite
# conn = sqlite3.connect('dados_descriptografados.db')
conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
df_descriptografado.to_sql('dados_descriptografados', conn, if_exists='replace', index=False)
conn.close()

/tmp/ipykernel_7377/2778535577.py:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_descriptografado = pd.read_json(mensagem_descriptografada)


In [14]:
conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
cursor = conn.cursor()

query = 'select * from dados_descriptografados'

cursor.execute(query)

resultados = cursor.fetchall()

# Exibir os resultados
for linha in resultados:

    print(linha)

(3500105, 'Adamantina', 35035, 'Adamantina', 3508, 'Presidente Prudente', 35, 'São Paulo', 'SP', 3, 'Sudeste', 'SE', 350019, 'Adamantina - Lucélia', 3505, 'Presidente Prudente', 35, 'SP', 'São Paulo', 3, 'SE', 'Sudeste', '2024-02-13')
(3500204, 'Adolfo', 35004, 'São José do Rio Preto', 3501, 'São José do Rio Preto', 35, 'São Paulo', 'SP', 3, 'Sudeste', 'SE', 350025, 'São José do Rio Preto', 3507, 'São José do Rio Preto', 35, 'SP', 'São Paulo', 3, 'SE', 'Sudeste', '2024-02-13')
(3500303, 'Aguaí', 35029, 'Pirassununga', 3507, 'Campinas', 35, 'São Paulo', 'SP', 3, 'Sudeste', 'SE', 350044, 'São João da Boa Vista', 3510, 'Campinas', 35, 'SP', 'São Paulo', 3, 'SE', 'Sudeste', '2024-02-13')
(3500402, 'Águas da Prata', 35030, 'São João da Boa Vista', 3507, 'Campinas', 35, 'São Paulo', 'SP', 3, 'Sudeste', 'SE', 350044, 'São João da Boa Vista', 3510, 'Campinas', 35, 'SP', 'São Paulo', 3, 'SE', 'Sudeste', '2024-02-13')
(3500501, 'Águas de Lindóia', 35033, 'Amparo', 3507, 'Campinas', 35, 'São Paul

In [ ]:
count_colum = (3500105, 
            'Adamantina',
            35035, 
            'Adamantina',
            3508,
            'Presidente Prudente',
            35,
            'São Paulo',
            'SP',
            3, 
            'Sudeste', 
            'SE', 
            350019, 
            'Adamantina - Lucélia',
            3505, 
            'Presidente Prudente',
            35, 
            'SP', 
            'São Paulo',
            3, 
            'SE', 
            'Sudeste',
            '2024-02-13')

In [22]:
import sqlite3

conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
cursor = conn.cursor()

create_silver = '''
CREATE TABLE IF NOT EXISTS teste_tabela_silver (
    id TEXT,
    nome TEXT,
    id_mun TEXT,
    nm_mun TEXT,
    id_mcr TEXT,
    nm_mcr TEXT,
    id_uf TEXT,
    nm_uf TEXT,
    sigla_uf TEXT,
    id_reg TEXT, 
    nm_reg TEXT,
    sigla_rg TEXT,
    rgimd_id TEXT,
    rgimd_nm TEXT,
    rgint_id TEXT,
    rgint_nm TEXT,
    rguf_id TEXT,
    rguf_sgl TEXT,
    rguf_nm TEXT,
    rgrg_id TEXT,
    rgrg_sgl TEXT,
    rgrg_nm TEXT,
    dtigtao DATE
);
'''

# Executar o script para criar a nova tabela
cursor.execute(create_silver)

# Commit para salvar as alterações
conn.commit()

# Fechar a conexão
conn.close()


In [12]:
cursor.execute("PRAGMA table_info(dados_descriptografados)")

# Recupere os resultados da consulta
colunas = cursor.fetchall()

# Imprima o nome das colunas
for coluna in colunas:
    print(coluna[1])

id
nome
id_mun
nm_mun
id_mcr
nm_mcr
id_uf
nm_uf
sigla_uf
id_reg
nm_reg
sigla_rg
rgimd_id
rgimd_nm
rgint_id
rgint_nm
rguf_id
rguf_sgl
rguf_nm
rgrg_id
rgrg_sgl
rgrg_nm
dtigtao


In [10]:
query = "select * from dados_descriptografados"
cursor.execute(query)

resultados = cursor.fetchall()

# Exibir os resultados
for linha in resultados:
    print(linha)

(3500105, 'Adamantina', 35035, 'Adamantina', 3508, 'Presidente Prudente', 35, 'São Paulo', 'SP', 3, 'Sudeste', 'SE', 350019, 'Adamantina - Lucélia', 3505, 'Presidente Prudente', 35, 'SP', 'São Paulo', 3, 'SE', 'Sudeste', '2024-02-13')
(3500204, 'Adolfo', 35004, 'São José do Rio Preto', 3501, 'São José do Rio Preto', 35, 'São Paulo', 'SP', 3, 'Sudeste', 'SE', 350025, 'São José do Rio Preto', 3507, 'São José do Rio Preto', 35, 'SP', 'São Paulo', 3, 'SE', 'Sudeste', '2024-02-13')
(3500303, 'Aguaí', 35029, 'Pirassununga', 3507, 'Campinas', 35, 'São Paulo', 'SP', 3, 'Sudeste', 'SE', 350044, 'São João da Boa Vista', 3510, 'Campinas', 35, 'SP', 'São Paulo', 3, 'SE', 'Sudeste', '2024-02-13')
(3500402, 'Águas da Prata', 35030, 'São João da Boa Vista', 3507, 'Campinas', 35, 'São Paulo', 'SP', 3, 'Sudeste', 'SE', 350044, 'São João da Boa Vista', 3510, 'Campinas', 35, 'SP', 'São Paulo', 3, 'SE', 'Sudeste', '2024-02-13')
(3500501, 'Águas de Lindóia', 35033, 'Amparo', 3507, 'Campinas', 35, 'São Paul

In [25]:
cursor.execute("DROP TABLE teste_tabela_silver")
print('Table drop')

Table drop


In [23]:
import sqlite3
from cryptography.fernet import Fernet

# Chave de criptografia (gerada apenas uma vez e mantida em segredo)
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Função para criptografar os dados
def encrypt_data(data):
    if isinstance(data, str):
        return cipher_suite.encrypt(data.encode()).decode()
    else:
        return data

# Conectar ao banco de dados SQLite
conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
cursor = conn.cursor()

# Selecionar todos os dados da tabela "dados_descriptografados"
cursor.execute("SELECT * FROM dados_descriptografados")
bronze_data = cursor.fetchall()

# Criptografar e transferir os dados para a tabela "teste_tabela_silver"
for row in bronze_data:
    # Criptografar os valores da linha
    encrypted_row = [encrypt_data(value) for value in row]
    
    # Inserir a linha criptografada na tabela "teste_tabela_silver"
    cursor.execute("INSERT INTO teste_tabela_silver VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", encrypted_row)

conn.commit()
conn.close()


In [17]:
import sqlite3
from cryptography.fernet import Fernet

# Gerar uma chave de criptografia
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Conectar ao banco de dados SQLite
conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
cursor = conn.cursor()

# Selecionar todos os dados da tabela "dados_descriptografados"
cursor.execute("SELECT * FROM dados_descriptografados")
bronze_data = cursor.fetchall()

# Criptografar e transferir os dados para a tabela "teste_tabela_silver"
for row in bronze_data:
    encrypted_row = []
    for value in row:
        if isinstance(value, str):  # Verifica se o valor é uma string antes de criptografá-lo
            encrypted_value = cipher_suite.encrypt(value.encode())
            encrypted_row.append(encrypted_value.decode())  # Decodificar para uma string antes de adicionar à lista
        else:
            encrypted_row.append(value)
    
    # Inserir a linha criptografada na tabela "teste_tabela_silver"
    cursor.execute("INSERT INTO teste_tabela_silver (id, nome, id_mun, nm_mun, id_mcr, nm_mcr, id_uf, nm_uf, sigla_uf, id_reg, nm_reg, sigla_rg, rgimd_id, rgimd_nm, rgint_id, rgint_nm, rguf_id, rguf_sgl, rguf_nm, rgrg_id, rgrg_sgl, rgrg_nm, dtigtao) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", encrypted_row)

conn.commit()
conn.close()


In [24]:
conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
cursor = conn.cursor()

query = 'select * from teste_tabela_silver'

cursor.execute(query)

resultados = cursor.fetchall()

# Exibir os resultados
for linha in resultados:

    print(linha)

('3500105', 'gAAAAABly5QSLrNB5sEBL6uD38CNjacCShh5c2MBiHIN7VEe2SoiAVfTEaGmXGjNK4pHq4a2mwA7i87TNiJgDmxbfbAIWXjypw==', '35035', 'gAAAAABly5QS4q7kyFyJcUK9FtwBcsr_QLa1P-YWFACZZ5gTHBgIel0k3Ik90SrZYI9wMSal8PQRc-MKWd6lcz--Zjis9EkJmg==', '3508', 'gAAAAABly5QSB-pX4d6BlPojhVRnSovmiHOUj7pECal3XCRSpVNeiixLNvLiC5KlOtebg5DgxJHJ7Hd_XR4z_e_bULypf5pd9JSPMnVO_1pCHDvM45-Axhg=', '35', 'gAAAAABly5QSjTvO8zkw6v2g8qoWNn-m3aoUN9NRICM0WDP4s0sX3-7VRrl4llgV2--lWZIZS3XdnSkTHfiG4KXvrlQBN_A11w==', 'gAAAAABly5QS4J_ikVJ4y_m9GLS2on4d1660WeH6JtQkNRXgedaDBWtfOd84VhYhU8_FnDOk5QeZz_X-wPF58vwOXTJRSKlbSg==', '3', 'gAAAAABly5QSB1phTT3humgAbOv4s8VfEIE_8je5QNYd4aSdnpHr4P9BTCJWPEeAzz5T9qhRrln5ZuAUuCd_wB9Rbu2QfT3HnA==', 'gAAAAABly5QSy4QmpJWfvXLlyZEYZZSc7fxkQ4LT9z-t7kxEdZj0g03HmikTDodpAp3ICKhhf-sq5BxHQcQGw-8L3dqKufbZwA==', '350019', 'gAAAAABly5QSvKUKyOP_oV8xeNX7tB93Sx17IbyvX93jl0B2sfftO90GHXkdJvsgH1oJd6uz0dbL-GwJMs01Rbn4IB5yNOiQjTlcUZMbGAciotEWuz1Xoeg=', '3505', 'gAAAAABly5QSGdquhUKTA_aWx6kFF_1WR1yYZ9pEhHdsoBOPeDYxm6_LGqmT4eD8wYN-J

In [16]:
import sqlite3
from cryptography.fernet import Fernet

# Gerar uma chave de criptografia
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Conectar ao banco de dados SQLite
# conn = sqlite3.connect('nome_do_banco_de_dados.db')
conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
cursor = conn.cursor()

# Selecionar todos os dados da tabela "bronze"
cursor.execute("SELECT * FROM dados_descriptografados")
bronze_data = cursor.fetchall()

# Criptografar e transferir os dados para a tabela "silver"
for row in bronze_data:
    encrypted_row = []
    for value in row:
        if isinstance(value, str):  # Verifica se o valor é uma string antes de criptografá-lo
            encrypted_value = cipher_suite.encrypt(value.encode())
            encrypted_row.append(encrypted_value.decode())  # Decodificar para uma string antes de adicionar à lista
        else:
            encrypted_row.append(value)
    
    # Inserir a linha criptografada na tabela "silver"
    cursor.execute("INSERT INTO teste_tabela_silver VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", encrypted_row)

conn.commit()
conn.close()


OperationalError: table teste_tabela_silver has 23 columns but 13 values were supplied

In [8]:
import sqlite3
from cryptography.fernet import Fernet

# Gerar uma chave de criptografia
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Conectar ao banco de dados SQLite
# conn = sqlite3.connect('nome_do_banco_de_dados.db')
conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
cursor = conn.cursor()

# Selecionar todos os dados da tabela "bronze"
cursor.execute("SELECT * FROM dados_descriptografados")
bronze_data = cursor.fetchall()

# Criptografar e transferir os dados para a tabela "silver"
for row in bronze_data:
    encrypted_row = []
    for value in row:
        if isinstance(value, str):  # Verifica se o valor é uma string antes de criptografá-lo
            encrypted_value = cipher_suite.encrypt(value.encode())
            encrypted_row.append(encrypted_value)
        else:
            encrypted_row.append(value)
    
    # Inserir a linha criptografada na tabela "silver"
    cursor.execute("INSERT INTO teste_tabela_silver VALUES (id, nome, idMunicipio, nomeMunicipio, idMacro, nomeMacroMunicipio, idUf, estado, siglaUf, idRegiao, regiao, siglaRegiao, dtIgtao)", encrypted_row)  # Substitua os pontos suspensos pelas colunas da tabela "silver"

conn.commit()
conn.close()


OperationalError: no such column: id

In [ ]:
import sqlite3
from cryptography.fernet import Fernet

# Gerar uma chave de criptografia
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Conectar ao banco de dados SQLite
# conn = sqlite3.connect('nome_do_banco_de_dados.db')
conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
cursor = conn.cursor()

# Normalizar os dados da tabela "bronze"
cursor.execute("SELECT * FROM dados_descriptografados")
bronze_data = cursor.fetchall()

# Processar e normalizar os dados conforme necessário

# Criptografar e transferir os dados para a tabela "silver"
for row in bronze_data:
    # Normalizar os dados conforme necessário
    
    # Aqui você normaliza os dados antes de criptografá-los
    
    # Por exemplo, se precisar criptografar o campo "nome" da tabela bronze:
    nome_normalizado = row[0]  # Supondo que o nome está na primeira coluna
    nome_criptografado = cipher_suite.encrypt(nome_normalizado.encode())
    
    # Agora, insira os dados normalizados e criptografados na tabela "silver"
    cursor.execute("INSERT INTO tabela_silver (nome_criptografado, outras_colunas) VALUES (?, ?)", (nome_criptografado, outras_colunas_normalizadas))
    
conn.commit()
conn.close()


In [8]:
import requests
import pandas as pd
from datetime import datetime
import cryptocode
import sqlite3

timestamp = datetime.now().strftime('%Y-%m-%d')

url_api = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados/35/municipios'

response = requests.get(url_api)

dados = response.json()

dados_list = []

if dados:
    for resultado in dados:
        id = resultado['id']
        nome = resultado['nome']
        id_mun = resultado['microrregiao']['id']
        nm_mun = resultado['microrregiao']['nome']
        id_mcr = resultado['microrregiao']['mesorregiao']['id']
        nm_mcr = resultado['microrregiao']['mesorregiao']['nome']
        id_uf = resultado['microrregiao']['mesorregiao']['UF']['id']
        nm_uf = resultado['microrregiao']['mesorregiao']['UF']['nome']
        sigla_uf = resultado['microrregiao']['mesorregiao']['UF']['sigla']
        id_reg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['id']
        nm_reg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['nome']
        sigla_rg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['sigla']
        rgimd_id = resultado['regiao-imediata']['id']
        rgimd_nm = resultado['regiao-imediata']['nome']
        rgint_id = resultado['regiao-imediata']['regiao-intermediaria']['id']
        rgint_nm = resultado['regiao-imediata']['regiao-intermediaria']['nome']
        rguf_id = resultado['regiao-imediata']['regiao-intermediaria']['UF']['id']
        rguf_sgl = resultado['regiao-imediata']['regiao-intermediaria']['UF']['sigla']
        rguf_nm = resultado['regiao-imediata']['regiao-intermediaria']['UF']['nome']
        rgrg_id = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['id']
        rgrg_sgl = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['sigla']
        rgrg_nm = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['nome']
        
        dados_list.append({
            'id': id,
            'nome': nome,
            'id_mun': id_mun,
            'nm_mun': nm_mun,
            'id_mcr': id_mcr,
            'nm_mcr': nm_mcr,
            'id_uf': id_uf,
            'nm_uf': nm_uf,
            'sigla_uf': sigla_uf,
            'id_reg': id_reg,
            'nm_reg': nm_reg,
            'sigla_rg': sigla_rg,
            'rgimd_id': rgimd_id,
            'rgimd_nm': rgimd_nm,
            'rgint_id': rgint_id,
            'rgint_nm': rgint_nm,
            'rguf_id': rguf_id,
            'rguf_sgl': rguf_sgl,
            'rguf_nm': rguf_nm,
            'rgrg_id': rgrg_id,
            'rgrg_sgl': rgrg_sgl,
            'rgrg_nm': rgrg_nm,
            'dtigtao': timestamp
        })

df = pd.DataFrame(dados_list)

# Converte o DataFrame em JSON e depois em uma string
mensagem = df.to_json()

chave = "423338233093093"

MensagemCriptografada = cryptocode.encrypt(mensagem, chave)
print("Sua mensagem criptografada: " + MensagemCriptografada)

# Salvar em um banco de dados SQLite
conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
cursor = conn.cursor()

# Crie uma tabela para armazenar os dados criptografados
cursor.execute('''CREATE TABLE IF NOT EXISTS dados_criptografados (
                    id INTEGER PRIMARY KEY,
                    mensagem TEXT
                )''')

# Insira a mensagem criptografada na tabela
cursor.execute("INSERT INTO dados_criptografados (mensagem) VALUES (?)", (MensagemCriptografada,))
conn.commit()

# Feche a conexão com o banco de dados
conn.close()


Sua mensagem criptografada: ...*9S/76jeq4DFgJClqeTcTTw==*gmEnGZlG+bj3u6bGjGIKpQ==*vLf7BJgGTe5tK/i1cinVDg==


In [9]:
conn = sqlite3.connect('/workspaces/api-geral/GEO/sql/teste_dados_criptografados.db')
cursor = conn.cursor()

query = 'select * from dados_criptografados'

cursor.execute(query)

resultados = cursor.fetchall()

# Exibir os resultados
for linha in resultados:
    print(linha)

(1, '...*9S/76jeq4DFgJClqeTcTTw==*gmEnGZlG+bj3u6bGjGIKpQ==*vLf7BJgGTe5tK/i1cinVDg==')


In [6]:
import requests
import pandas as pd
from datetime import datetime
import cryptocode
import json

timestamp = datetime.now().strftime('%Y-%m-%d')

url_api = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados/35/municipios'

response = requests.get(url_api)

dados = response.json()

dados_list = []


if dados:
    for resultado in dados:
        id = resultado['id']
        nome = resultado['nome']
        id_mun = resultado['microrregiao']['id']
        nm_mun = resultado['microrregiao']['nome']
        id_mcr = resultado['microrregiao']['mesorregiao']['id']
        nm_mcr = resultado['microrregiao']['mesorregiao']['nome']
        id_uf = resultado['microrregiao']['mesorregiao']['UF']['id']
        nm_uf = resultado['microrregiao']['mesorregiao']['UF']['nome']
        sigla_uf = resultado['microrregiao']['mesorregiao']['UF']['sigla']
        id_reg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['id']
        nm_reg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['nome']
        sigla_rg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['sigla']
        rgimd_id = resultado['regiao-imediata']['id']
        rgimd_nm = resultado['regiao-imediata']['nome']
        rgint_id = resultado['regiao-imediata']['regiao-intermediaria']['id']
        rgint_nm = resultado['regiao-imediata']['regiao-intermediaria']['nome']
        rguf_id = resultado['regiao-imediata']['regiao-intermediaria']['UF']['id']
        rguf_sgl = resultado['regiao-imediata']['regiao-intermediaria']['UF']['sigla']
        rguf_nm = resultado['regiao-imediata']['regiao-intermediaria']['UF']['nome']
        rgrg_id = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['id']
        rgrg_sgl = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['sigla']
        rgrg_nm = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['nome']
        
        dados_list.append({
            'id': id,
            'nome': nome,
            'id_mun': id_mun,
            'nm_mun': nm_mun,
            'id_mcr': id_mcr,
            'nm_mcr': nm_mcr,
            'id_uf': id_uf,
            'nm_uf': nm_uf,
            'sigla_uf': sigla_uf,
            'id_reg': id_reg,
            'nm_reg': nm_reg,
            'sigla_rg': sigla_rg,
            'rgimd_id': rgimd_id,
            'rgimd_nm': rgimd_nm,
            'rgint_id': rgint_id,
            'rgint_nm': rgint_nm,
            'rguf_id': rguf_id,
            'rguf_sgl': rguf_sgl,
            'rguf_nm': rguf_nm,
            'rgrg_id': rgrg_id,
            'rgrg_sgl': rgrg_sgl,
            'rgrg_nm': rgrg_nm,
            'dtigtao': timestamp
        })


df = pd.DataFrame(dados_list)

# Converte o DataFrame em JSON e depois em uma string
mensagem = df.to_json()

chave = "423338233093093"

MensagemCriptografada = cryptocode.encrypt(mensagem, chave)
print("Sua mensagem criptografada: " + MensagemCriptografada)

MensagemDescriptografada = cryptocode.decrypt(MensagemCriptografada, chave)

# Convertendo a mensagem descriptografada de volta para DataFrame
df_descriptografado = pd.read_json(MensagemDescriptografada)
print(df_descriptografado)


Sua mensagem criptografada: ...*0PoSEu4Vf5hgkgCsGq2SRQ==*54rRSvfHHqmFHaYtytoXWQ==*Z7ZCtJ0Y27UQ+qZvFXY4kg==
          id              nome  id_mun                 nm_mun  id_mcr  \
0    3500105        Adamantina   35035             Adamantina    3508   
1    3500204            Adolfo   35004  São José do Rio Preto    3501   
2    3500303             Aguaí   35029           Pirassununga    3507   
3    3500402    Águas da Prata   35030  São João da Boa Vista    3507   
4    3500501  Águas de Lindóia   35033                 Amparo    3507   
..       ...               ...     ...                    ...     ...   
640  3557006        Votorantim   35046               Sorocaba    3512   
641  3557105       Votuporanga   35003            Votuporanga    3501   
642  3557154          Zacarias   35004  São José do Rio Preto    3501   
643  3557204         Chavantes   35040               Ourinhos    3510   
644  3557303      Estiva Gerbi   35031             Mogi Mirim    3507   

                

/tmp/ipykernel_7377/2088974008.py:83: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_descriptografado = pd.read_json(MensagemDescriptografada)


In [5]:
from cryptography.fernet import Fernet 
import requests
import pandas as pd
from datetime import datetime
import cryptocode

timestamp = datetime.now().strftime('%Y-%m-%d')

url_api = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados/35/municipios'

response = requests.get(url_api)

dados = response.json()

dados_list = []


if dados:
    for resultado in dados:
        id = resultado['id']
        nome = resultado['nome']
        id_mun = resultado['microrregiao']['id']
        nm_mun = resultado['microrregiao']['nome']
        id_mcr = resultado['microrregiao']['mesorregiao']['id']
        nm_mcr = resultado['microrregiao']['mesorregiao']['nome']
        id_uf = resultado['microrregiao']['mesorregiao']['UF']['id']
        nm_uf = resultado['microrregiao']['mesorregiao']['UF']['nome']
        sigla_uf = resultado['microrregiao']['mesorregiao']['UF']['sigla']
        id_reg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['id']
        nm_reg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['nome']
        sigla_rg = resultado['microrregiao']['mesorregiao']['UF']['regiao']['sigla']
        rgimd_id = resultado['regiao-imediata']['id']
        rgimd_nm = resultado['regiao-imediata']['nome']
        rgint_id = resultado['regiao-imediata']['regiao-intermediaria']['id']
        rgint_nm = resultado['regiao-imediata']['regiao-intermediaria']['nome']
        rguf_id = resultado['regiao-imediata']['regiao-intermediaria']['UF']['id']
        rguf_sgl = resultado['regiao-imediata']['regiao-intermediaria']['UF']['sigla']
        rguf_nm = resultado['regiao-imediata']['regiao-intermediaria']['UF']['nome']
        rgrg_id = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['id']
        rgrg_sgl = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['sigla']
        rgrg_nm = resultado['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['nome']
        
        
        dados_list.append({
            'id': id,
            'nome': nome,
            'id_mun': id_mun,
            'nm_mun': nm_mun,
            'id_mcr': id_mcr,
            'nm_mcr': nm_mcr,
            'id_uf': id_uf,
            'nm_uf': nm_uf,
            'sigla_uf': sigla_uf,
            'id_reg': id_reg,
            'nm_reg': nm_reg,
            'sigla_rg': sigla_rg,
            'rgimd_id': rgimd_id,
            'rgimd_nm': rgimd_nm,
            'rgint_id': rgint_id,
            'rgint_nm': rgint_nm,
            'rguf_id': rguf_id,
            'rguf_sgl': rguf_sgl,
            'rguf_nm': rguf_nm,
            'rgrg_id': rgrg_id,
            'rgrg_sgl': rgrg_sgl,
            'rgrg_nm': rgrg_nm,
            'dtigtao': timestamp
            
        })


df = pd.DataFrame(dados_list)
# print(df)




chave = "423338233093093"
mensagem = df

MensagemCriptografada = cryptocode.encrypt(mensagem, chave)
print("Sua mensagem criptografada: " + MensagemCriptografada)

MensagemDescriptografada = cryptocode.decrypt(MensagemCriptografada, chave)
print("Sua mensagem descriptografada: " + MensagemDescriptografada)








# message = df
 
# key = Fernet.generate_key() 
  
  
# fernet = Fernet(key) 
  
# encMessage = fernet.encrypt(message.encode()) 
  
# print("original string: ", message) 
# print("encrypted string: ", encMessage) 
  
# decMessage = fernet.decrypt(encMessage).decode() 
  
# print("decrypted string: ", decMessage) 


# file = f'dados_geo_{sigla_uf}.txt'
# path = '/workspaces/api-geral/GEO/dados/'

# df.to_csv(f'{path}{file}', sep=';', index=False)
# print(f'Arquivo salvo: {file}')
# print(f'Caminho: {path}')

TypeError: encoding without a string argument